#  CS 247 : Advanced Data Mining Learning
## Homework 6

### Due: 11:59 pm 03/02

##### Please read the Homework Guidance (uploaded to BruinLearn) carefully and make sure you fulfill all the requirements.



## Problem 1: Graph Convolutional Network (50 pts = 20 + 20 + 10)


In this problem, you are going to do node classification on Cora dataset with graph convolutional network. The `Cora` network holds 2,708 nodes and 10,556 edges, resulting in an average node degree of 3.9.
For training this dataset, we are given the ground-truth categories of 140 nodes (20 for each class).

You are going to implement a __two layer GCN model__ with the __GCNConv__ layer provided by __torch_geometric__. 

Complete the code and get the test accuracy. You can use whichever activation function, change all the hyper-parameters (__except the number of layers__). The expected test accuracy should be larger than $0.8$.

__Hint__:

1. You can use *torch.manual_seed()* to remove randomness

2. See https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv for a brief introduction to GCNConv. 

In [1]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 14.5 MB/s 
     |████████████████████████████████| 3.5 MB 12.5 MB/s 


In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

# Download the dataset

dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Processing...
Done!


In [6]:
from torch_geometric.nn import GCNConv
import torch
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super().__init__()
        """
          TODO: implement the initialzation of the GCN model.
            It should contain two GCNConv layers.
        """


        self.gcn_l1 = GCNConv(num_features,hidden_channels)
        self.gcn_l2 = GCNConv(hidden_channels,num_classes)
        self.rel = torch.nn.ReLU()
    
        

    def forward(self, x, edge_index):
      x = self.gcn_l1(x,edge_index)
      x = self.rel(x)
      
      x = self.gcn_l2(x,edge_index)


    
      
      return x

      """
        TODO: implement the forward pass of the GCN model.
      """
       

In [7]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(num_features = dataset.num_features, hidden_channels=16,
            num_classes=dataset.num_classes)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      # print(data.y,"fds")
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      """
        TODO: implement the code to compute loss function
          You should use the 'criterion' we defined above              
      """
      # Compute the loss solely based on the training nodes.
      
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 150):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

GCN(
  (gcn_l1): GCNConv(1433, 16)
  (gcn_l2): GCNConv(16, 7)
  (rel): ReLU()
)
Epoch: 001, Loss: 1.9457
Epoch: 002, Loss: 1.9412
Epoch: 003, Loss: 1.9329
Epoch: 004, Loss: 1.9216
Epoch: 005, Loss: 1.9097
Epoch: 006, Loss: 1.8980
Epoch: 007, Loss: 1.8854
Epoch: 008, Loss: 1.8718
Epoch: 009, Loss: 1.8570
Epoch: 010, Loss: 1.8411
Epoch: 011, Loss: 1.8247
Epoch: 012, Loss: 1.8075
Epoch: 013, Loss: 1.7896
Epoch: 014, Loss: 1.7710
Epoch: 015, Loss: 1.7517
Epoch: 016, Loss: 1.7317
Epoch: 017, Loss: 1.7112
Epoch: 018, Loss: 1.6901
Epoch: 019, Loss: 1.6683
Epoch: 020, Loss: 1.6460
Epoch: 021, Loss: 1.6233
Epoch: 022, Loss: 1.6000
Epoch: 023, Loss: 1.5762
Epoch: 024, Loss: 1.5520
Epoch: 025, Loss: 1.5274
Epoch: 026, Loss: 1.5024
Epoch: 027, Loss: 1.4770
Epoch: 028, Loss: 1.4513
Epoch: 029, Loss: 1.4253
Epoch: 030, Loss: 1.3991
Epoch: 031, Loss: 1.3727
Epoch: 032, Loss: 1.3463
Epoch: 033, Loss: 1.3197
Epoch: 034, Loss: 1.2931
Epoch: 035, Loss: 1.2666
Epoch: 036, Loss: 1.2401
Epoch: 037, Loss: 1.

In [9]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.8090


## Problem 2: Knowledge Graph Embedding (50 pts)

In this problem, you are going to consider the TransE, DistMult, and RotatE models (slide09 P23-30). You are going to answer the following questions and provide reasons.

### Part 1 (20 pts)

If we have a knowledge graph with __friendship__ and __enemy__ relationship, which model(s) of the TransE, DistMult, and RotatE we __can__ use? Please explain your reason based on the score function of each model.

__Hint__

Friendship and enemy are symmetric relationships.

#### Write Your Answer Here

![pic](https://drive.google.com/uc?export=view&id=1pwrvElFdVTrqsRQrYIMkk3BR6oYcpdvz)
![pic](https://drive.google.com/uc?export=view&id=1lqxaEFSiwb-fsCVEDe758ou8kNz1UJ-K)
![pic](https://drive.google.com/uc?export=view&id=1Kya-dyKmJfSGb4yxwEROPSepexMk7ipH)

### Part 2 (10 pts)

If we have a graph with the __teaching__ relationship, which model(s) we __should not__ use? Please explain your reason based on the score function.

__Hint__

Teaching is an asymmetric relationship.

#### Write Your Answer Here


![pic](https://drive.google.com/uc?export=view&id=1Wzck2Njmg3sDHUwXSRRM_bvozJNkBCSw)
![pic](https://drive.google.com/uc?export=view&id=1_wOkW1jVjxIXmOh0fNnrlhTiKXJ4HvQw)
![pic](https://drive.google.com/uc?export=view&id=1Uz-cW5k-mU_6qrd6DYm1X1EpRb6t-NU9)

## Part 3 (20 pts)

If we have a knowledge graph with __father, grandfather, mother, and grandmother__ relationship, which model(s) we __can__ use? Please explain your reason based on the score function.

__Hint__

The father of father is grandfather. The mother of mother is grandmother. Which model(s) can model composition relationship? How？

#### Write Your Answer Here



![pic](https://drive.google.com/uc?export=view&id=1v285fbzAmTWnDNYdi4yl5gAianH6d-Hd)
![pic](https://drive.google.com/uc?export=view&id=1KNgX2e90gty4MaUEt8j4MSEqT0CUSrZ4)
![pic](https://drive.google.com/uc?export=view&id=1wJkmb8jQOcdMtPGlaNao4CjtReVdZJXZ)

## Bonus Problem: Relational GCN (10 pts)

The knowledge graph embedding models taught in the lecture are all shallow embeddings. And we learned that a typical deep embedding model for graphs is GCN, which, according to the slides, can be describe as
$$
h_v^k = \sigma(W_k\sum_{u\in N(v)⋃ v}\frac{h_u^{k-1}}{\sqrt{|N(u)||N(v)|}})
$$

However, it cannot handle knowledge graphs because it does not consider edge labels in its model.

Please describe a way to extend the GCN model so it can handle knowledge graph. 

#### Write Your Answer Here